## *Setup required for Playground*

The online playground requires a little bit of code to run first.

In [ ]:
import piplite
await piplite.install('kloppy')
await piplite.install('pandas')

import pyodide_http

# State

While working with event data it can be convenient to also use the current score. This can be used to determine if a team is winning, losing or drawing. A more generic name for the score is 'state'. 

In this quickstart we'll look at how to use the `add_state` method of kloppy to add state to the events for later use.

## Loading some statsbomb data

First we'll load Barcelona - Deportivo Alaves from the statsbomb open-data project.

In [ ]:
from kloppy import statsbomb
from kloppy.domain import EventType

dataset = statsbomb.load_open_data()
print([team.name for team in dataset.metadata.teams])
print(dataset.events[0].state)

## Add state - score

kloppy contains some default state builders: score, lineup and sequence. Let's have a look at the `score` state builder. 

In [ ]:
dataset = dataset.add_state('score')

dataset.events[0].state

As you can see `state` is now filled with a score object. The object contains two attributes: `home` and `away`. Every event contains a score object which is automatically updated when a goal is scored.

Now lets have a look at how we can use the score state. First we filter on only shots.

In [ ]:
dataset = dataset.filter(lambda event: event.event_type == EventType.SHOT)
shots = dataset.events
len(shots)

In [ ]:
for shot in shots:
    print(shot.state['score'], shot.player.team, '-', shot.player, '-', shot.result)

In [ ]:
dataframe = dataset.to_df(
    "*",
    home_score=lambda event: event.state['score'].home,
    away_score=lambda event: event.state['score'].away
)
dataframe

Now filter the dataframe. We only want to see shots when we are winning by at least two goals difference.

In [ ]:
dataframe[dataframe['home_score'] - dataframe['away_score'] >= 2]

## Add state - lineup

We are able to add more state. In this example we'll look at adding lineup state.

In [ ]:
from kloppy import statsbomb
from kloppy.domain import EventType

dataset = statsbomb.load_open_data()

home_team, away_team = dataset.metadata.teams

Arturo Vidal is a substitute on the side of Barcelona. We add lineup to all events so we are able to filter out events where Arturo Vidal is on the pitch.

In [ ]:
arturo_vidal = home_team.get_player_by_id(8206)

In [ ]:
dataframe = (
    dataset
    .add_state('lineup')
    .filter(lambda event: arturo_vidal in event.state['lineup'].players)
    .to_df()
)

In [ ]:
print(f"time on pitch: {dataframe['timestamp'].max() - dataframe['timestamp'].min()} seconds")


In [ ]:
dataframe = (
    dataset
    .add_state('lineup')
    .filter(lambda event: event.event_type == EventType.PASS and event.team == home_team)
    .to_df(
        "*",
        vidal_on_pitch=lambda event: arturo_vidal in event.state['lineup'].players
    )
)

In [ ]:
dataframe = dataframe.groupby(['vidal_on_pitch'])['success'].agg(['sum', 'count'])
dataframe['percentage'] = dataframe['sum'] / dataframe['count'] * 100
dataframe

## Add state - formation

In this example we'll look at adding formation state to all shots.

In [ ]:
from kloppy import statsbomb
from kloppy.domain import EventType


dataset = statsbomb.load_open_data()

dataframe = (
    dataset
    .add_state('formation')
    .filter(
        lambda event: event.event_type == EventType.SHOT
    )
    .to_df(
        "*",
        Team=lambda event: str(event.team),
        Formation=lambda event: str(
            event.state['formation'].home 
            if event.team == dataset.metadata.teams[0] 
            else event.state['formation'].away
        )
    )
)

In [ ]:
dataframe_stats = (
    dataframe
    .groupby(['Team', 'Formation'])['success']
    .agg(['sum', 'count'])
)
dataframe_stats['Percentage'] = (
    dataframe_stats['sum'] 
    / dataframe_stats['count'] 
    * 100
)
dataframe_stats.rename(
    columns={
        'sum': 'Goals',
        'count': 'Shots'
    }
)